In [1]:
import os
import pytorch_lightning as pl
import pandas as pd

from rdkit import Chem # safe import before ccdc imports
from torch_geometric.loader import DataLoader
from torch.utils.data import ConcatDataset

from conf_ensemble_dataset_in_memory import ConfEnsembleDataset
from litschnet import LitSchNet
from molsize_model import MolSizeModel
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm import tqdm

In [2]:
pl.seed_everything(42, workers=True)

Global seed set to 42


42

# Data preparation

In [3]:
# run once to preprocess datasets and generate chunks
# dataset = ConfEnsembleDataset()
# dataset = ConfEnsembleDataset(dataset='platinum')

Processing...
  1%|▍                                        | 51/4548 [00:32<59:16,  1.26it/s]RDKit ERROR: [17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
RDKit ERROR: [17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
RDKit ERROR: [17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
RDKit ERROR: [17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
[17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
RDKit ERROR: [17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
[17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
[17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
[17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
[17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
RDKit ERROR: [17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
RDKit ERROR: [17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
[17:54:10] UFFTYPER: Unrecognized charge state for atom: 13
[17:54:10] UFFTYPER: Unrecognized c

In [4]:
splits = ['random', 'scaffold', 'protein']

In [5]:
def get_loaders(split, 
                iteration, 
                data_dir='data/',
                ) :
    
    pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
    #pdbbind_n_chunks = len(pdbbind_chunks)
    pdbbind_n_chunks = 4
    
    train_datasets = []
    val_datasets = []
    test_datasets = []
    
    if split in ['random', 'scaffold'] :
        
        with open(os.path.join(data_dir, f'ligand_{split}_splits', f'train_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            train_smiles = f.readlines()
            train_smiles = [smiles.strip() for smiles in train_smiles]

        with open(os.path.join(data_dir, f'ligand_{split}_splits', f'val_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            val_smiles = f.readlines()
            val_smiles = [smiles.strip() for smiles in val_smiles]

        with open(os.path.join(data_dir, f'ligand_{split}_splits', f'test_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            test_smiles = f.readlines()
            test_smiles = [smiles.strip() for smiles in test_smiles]

        for chunk_number in tqdm(range(pdbbind_n_chunks)) :

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          smiles_list=train_smiles)
            train_datasets.append(dataset)

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          smiles_list=val_smiles)
            val_datasets.append(dataset)

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          smiles_list=test_smiles)
            test_datasets.append(dataset)
            
    else : #protein split
        
        with open(os.path.join(data_dir, 'protein_similarity_splits', f'train_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
            train_pdbs = f.readlines()
            train_pdbs = [pdb.strip() for pdb in train_pdbs]

        with open(os.path.join(data_dir, 'protein_similarity_splits', f'val_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
            val_pdbs = f.readlines()
            val_pdbs = [pdb.strip() for pdb in val_pdbs]

        with open(os.path.join(data_dir, 'protein_similarity_splits', f'test_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
            test_pdbs = f.readlines()
            test_pdbs = [pdb.strip() for pdb in test_pdbs]

        for chunk_number in tqdm(range(pdbbind_n_chunks)) :

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          pdb_ids_list=train_pdbs)
            train_datasets.append(dataset)

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          pdb_ids_list=val_pdbs)
            val_datasets.append(dataset)

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          pdb_ids_list=test_pdbs)
            test_datasets.append(dataset)

    train_dataset = ConcatDataset(train_datasets)
    val_dataset = ConcatDataset(val_datasets)
    test_dataset = ConcatDataset(test_datasets)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)
    
    return train_loader, val_loader, test_loader

In [6]:
for split in splits :
    
    for iteration in range(5) :
    
        train_loader, val_loader, test_loader = get_loaders(split, iteration)
        
        experiment_name = f'{split}_split_{iteration}'
        if not experiment_name in os.listdir('lightning_logs') :
            litschnet = LitSchNet()
            logger = TensorBoardLogger(save_dir=os.getcwd(), version=experiment_name, name="lightning_logs")
            trainer = pl.Trainer(logger=logger, callbacks=[EarlyStopping(monitor="val_loss", patience=5)], gpus=1)
            trainer.fit(litschnet, train_loader, val_loader)
            trainer.test(litschnet, test_loader)
            
        experiment_name = f'{split}_split_{iteration}_molsize'
        if not experiment_name in os.listdir('lightning_logs') :
            molsize_model = MolSizeModel()
            logger = TensorBoardLogger(save_dir=os.getcwd(), version=experiment_name, name="lightning_logs")
            trainer = pl.Trainer(logger=logger, callbacks=[EarlyStopping(monitor="val_loss", patience=5)], gpus=1)
            trainer.fit(molsize_model, train_loader, val_loader)
            trainer.test(molsize_model, test_loader)

100%|█████████████████████████████████████████████| 4/4 [04:33<00:00, 68.40s/it]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | linear_layers | Sequential | 10.5 K
1 | leaky_relu    | LeakyReLU  | 0     
---------------------------------------------
10.5 K    Trainable params
0         Non-trainable params
10.5 K    Total params
0.042     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:57: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 10. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.48515915870666504}
--------------------------------------------------------------------------------


In [6]:
from collections import defaultdict
n_bioactive_conformations = defaultdict(list)
n_conformations = defaultdict(list)
data_dir = 'data/'

for split in splits :
    
    for iteration in range(5) :
    
        pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
        #pdbbind_n_chunks = len(pdbbind_chunks)
        pdbbind_n_chunks = 4

        train_datasets = []
        val_datasets = []
        test_datasets = []

        if split in ['random', 'scaffold'] :

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'train_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                train_smiles = f.readlines()
                train_smiles = [smiles.strip() for smiles in train_smiles]

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'val_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                val_smiles = f.readlines()
                val_smiles = [smiles.strip() for smiles in val_smiles]

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'test_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                test_smiles = f.readlines()
                test_smiles = [smiles.strip() for smiles in test_smiles]

            for chunk_number in tqdm(range(pdbbind_n_chunks)) :

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=train_smiles)
                train_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=val_smiles)
                val_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=test_smiles)
                test_datasets.append(dataset)

        else : #protein split

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'train_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                train_pdbs = f.readlines()
                train_pdbs = [pdb.strip() for pdb in train_pdbs]

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'val_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                val_pdbs = f.readlines()
                val_pdbs = [pdb.strip() for pdb in val_pdbs]

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'test_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                test_pdbs = f.readlines()
                test_pdbs = [pdb.strip() for pdb in test_pdbs]

            for chunk_number in tqdm(range(pdbbind_n_chunks)) :

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=train_pdbs)
                train_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=val_pdbs)
                val_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=test_pdbs)
                test_datasets.append(dataset)

        train_dataset = ConcatDataset(train_datasets)
        val_dataset = ConcatDataset(val_datasets)
        test_dataset = ConcatDataset(test_datasets)
        
        d = {
            'train' : train_dataset,
            'val' : val_dataset,
            'test' : test_dataset
        }
        for s, dataset in d.items() :
            n_conformations[s].append(len(dataset))
            n_bio = 0
            for data in dataset :
                if data.rmsd == 0 :
                    n_bio = n_bio + 1
            n_bioactive_conformations[s].append(n_bio)

100%|████████████████████████████████████████████| 4/4 [07:28<00:00, 112.15s/it]


In [7]:
n_conformations

defaultdict(list,
            {'train': [663311,
              660776,
              664804,
              663873,
              663678,
              641347,
              644081,
              643457,
              647182,
              641304,
              647812,
              656059,
              654465,
              652408,
              664808],
             'val': [92502,
              90048,
              92768,
              91319,
              90925,
              83061,
              76645,
              80580,
              79627,
              77341,
              91794,
              86286,
              87041,
              89268,
              86884],
             'test': [90702,
              94593,
              89409,
              91563,
              89860,
              80214,
              83896,
              80585,
              77813,
              85977,
              92624,
              89541,
              93168,
              89448,
              830

In [8]:
n_bioactive_conformations

defaultdict(list,
            {'train': [10683,
              10664,
              10704,
              10706,
              10684,
              9978,
              10087,
              9999,
              10136,
              10089,
              10534,
              10666,
              10669,
              10601,
              10655],
             'val': [1915,
              1916,
              1928,
              1828,
              1914,
              1268,
              1282,
              1281,
              1215,
              1263,
              1426,
              1333,
              1270,
              1399,
              1400],
             'test': [1881,
              1893,
              1907,
              1960,
              1861,
              1316,
              1193,
              1282,
              1211,
              1210,
              1386,
              1349,
              1413,
              1350,
              1296]})